Collection of tried methods and commands


##### CUDA GPU installation confirmation, extensive setup tested for potential better image processing but GPU memory proved insufficient and led to frequent crashes. 

In [3]:
import tensorflow as tf 
print(tf.test.is_built_with_cuda()) 
print(tf.config.list_physical_devices('GPU'))

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


##### First simple CNN model adding layers from scratch to get benchmark:

In [16]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(num_classes, activation='softmax'))


In [17]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_generator,
          epochs=5,
          validation_data=validation_generator)          
#validation_data=validation_generator

Epoch 1/5
188/188 [==============================] - ETA: 0s - loss: 2.8402 - accuracy: 0.0743

C:\Users\ShwetankPC\anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:2847: DecompressionBombWarning: Image size (95799284 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


188/188 [==============================] - 683s 4s/step - loss: 2.8402 - accuracy: 0.0743 - val_loss: 4.4883 - val_accuracy: 0.0000e+00
Epoch 2/5
188/188 [==============================] - 580s 3s/step - loss: 2.6567 - accuracy: 0.1128 - val_loss: 5.9962 - val_accuracy: 0.0000e+00
Epoch 3/5
188/188 [==============================] - 580s 3s/step - loss: 2.6065 - accuracy: 0.1330 - val_loss: 6.7849 - val_accuracy: 0.0000e+00
Epoch 4/5
188/188 [==============================] - 575s 3s/step - loss: 2.5572 - accuracy: 0.1511 - val_loss: 7.8191 - val_accuracy: 0.0000e+00
Epoch 5/5
188/188 [==============================] - 578s 3s/step - loss: 2.4925 - accuracy: 0.1824 - val_loss: 8.5285 - val_accuracy: 0.0000e+00


Note, the validation accuracy is showing as zero because the validation set had not been properly shuffled before split and so none of its classes were being recognized. However, the training accuracy could still be used to compare models.

In [ ]:
model.fit(train_generator,
          epochs=5,
          validation_data=validation_generator)    

In [41]:
score = model.evaluate(train_generator, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

188/188 [==============================] - 420s 2s/step - loss: 2.4452 - accuracy: 0.2081
Test loss: 2.4452273845672607
Test accuracy: 0.20805548131465912


A decent test accuracy and the model could certainly be kept going for training, however, other pre-trained methods seemed to rise faster in accuracy.

##### Art style classification first attempt with ResNet (using flatten and two ReLu layers instead of the two better performing pooling layers used in 'Capstone_StyleClassification' notebook): 

In [50]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2 

model2 = ResNet50V2 (weights='imagenet',
                   include_top=False,
                   input_shape=(height,width,3))

for layer in model2.layers:
    layer.trainable = False



    # Add our own layers to the end of the network
x = model2.output 
x = Flatten()(x) 
x = Dense(200, activation='relu')(x)
x = Dense(50, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model2 = Model(inputs=model2.input, 
              outputs=output)

In [51]:
model2.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model2.fit(train_generator,
          epochs=2,
          #validation_split = 0.25,
          validation_data=validation_generator)

Epoch 1/2
263/263 [==============================] - 1051s 4s/step - loss: 3.0618 - accuracy: 0.1495 - val_loss: 5.3718 - val_accuracy: 0.0031
Epoch 2/2
263/263 [==============================] - 954s 4s/step - loss: 2.4606 - accuracy: 0.2125 - val_loss: 7.4308 - val_accuracy: 0.0000e+00


Accuracy rises much faster than from scratch model above but was slower than the Xception model. 

#### Setup for NASNetMobile (mobile version having very few parameters), while producing decent accuracy gains, it was slower than Xception and ResNet and so was abandoned:

In [17]:
from tensorflow.keras.applications import NASNetMobile 

model2 = NASNetMobile (weights='imagenet',
                   include_top=False,
                   input_shape=(height,width,3))

for layer in model2.layers:
    layer.trainable = False

    # Add our own layers to the end of the network
x = model2.output 
x = Flatten()(x) 
x = Dense(500, activation='relu')(x)
#x = Dense(150, activation='relu')(x)
output = Dense(126, activation='softmax')(x)

model2 = Model(inputs=model2.input, 
              outputs=output)

19996672/19993432 [==============================] - 2s 0us/step


In [ ]:
model2.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Use an early stopping callback to stop training
# once we no longer have improvements in our validation loss
model2.fit(train_generator,
          epochs=5,
          #validation_split = 0.25,
          validation_data=validation_generator,
          callbacks=[early_stop])

Epoch 1/5
256/256 [==============================] - ETA: 0s - loss: 4.7792 - accuracy: 0.1623

C:\Users\ShwetankPC\anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:2847: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


256/256 [==============================] - 460s 2s/step - loss: 4.7792 - accuracy: 0.1623 - val_loss: 3.3927 - val_accuracy: 0.1799
Epoch 2/5
256/256 [==============================] - 430s 2s/step - loss: 2.7140 - accuracy: 0.2897 - val_loss: 3.4230 - val_accuracy: 0.1774
Epoch 3/5
256/256 [==============================] - ETA: 0s - loss: 2.2148 - accuracy: 0.3917

#### Setup for EfficientNetB0 testing, results were deleted during course of trial and error, remained very low below 10% for first 6 epochs and so was abandoned:

In [59]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

    
modelb = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(height,width,3))
    # Freeze the pretrained weights
modelb.trainable = False

    # Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(modelb.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
x = Flatten()(x) 
#x = Dense(1000, activation='relu')(x)
x = Dense(400, activation='relu')(x)
x = Dense(100, activation='relu')(x)
outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
modelb = Model(modelb.input, outputs, name="EfficientNet")
optimizer = Adam(learning_rate=1e-2)
modelb.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    

In [ ]:
epochs = 10  # @param {type: "slider", min:8, max:80}
hist = modelb.fit(train_generator, epochs=epochs, validation_data=validation_generator, verbose=1)
plot_hist(hist)

In [ ]:
modelb.summary()

##### Demonstration of loading a model testing accuracy on test dataset where it works, not sure why it works here but did not in 'Captstone_ArtistClassification' file when doing same for VGG model

In [63]:
from tensorflow.keras.models import load_model
model=load_model('D:\Documents\BrainStation\Capstone\Res_best')

In [80]:
testpath = r'D:\Documents\BrainStation\Capstone\test\test\\'

#train_limit=int(len(samples)*0.75)
test_gen=ImageDataGenerator(rescale=1./255.)

height=224
width=224

test_generator=test_gen.flow_from_dataframe(
dataframe=dftest,
directory=testpath,
x_col="new_filename",
y_col="style",
#subset="validation",
color_mode='rgb',
batch_size=40,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(height,width))


Found 18697 validated image filenames belonging to 20 classes.


In [85]:
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.models import Model

testmodel= Model(inputs=model.input, 
              outputs=model.output)

metric = TopKCategoricalAccuracy(k=3)
testmodel.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=metric)

testmodel.evaluate(test_generator)

468/468 [==============================] - 1097s 2s/step - loss: 2.5165 - top_k_categorical_accuracy: 0.5859


[2.5164759159088135, 0.58592289686203]

##### Some of the operations carried out to reduce number of style classes and possibly increase accuracy (did not have any significant effect)

In [ ]:
df['style'].replace('Post-Impressionism', 'Impressionism', inplace=True)     #Combine two styles together
df['style'].replace('Mannerism (Late Renaissance)', 'Renaissance', inplace=True)
df['style'].replace('Early Renaissance', 'Renaissance', inplace=True)
df['style'].replace('High Renaissance', 'Renaissance', inplace=True)
df['style'].replace('Northern Renaissance', 'Renaissance', inplace=True)
df['style'].replace('Academicism', 'Neoclassicism', inplace=True
                    
styles=styles.drop(['Art Informel', 'Abstract Expressionism', 'Abstract Art', 'Color Field Painting'])`     #Drop abstract type art 